This notebook focuses on writing code that simulates what recommendations would look like for songs in a playlist. This is recommender is working with the generic (kaggle) dataset and is not using the personalised one made in notebook 03_Personalized_Recommender.

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from numpy import dot
from numpy.linalg import norm

from sklearn.preprocessing import StandardScaler

Functions from first recommeder notebook will be used. As well as importing the necessary dataframe as it is required for the functions to work.

## Import Datasets

In [2]:
#import data
data_df = pd.read_csv('../data/spotify_songs_1922/cleaned_data.csv')

### create neccesary dataframes

In [3]:
#create dataframe only used to apply cosine similarity calculation on
# only take the numerical columns and drop the numerical ones that do not represent audio features
data_df_cosim = data_df.select_dtypes(include=['int64','float64']).drop(columns=['year','popularity','explicit'])

#concat with id so that we can use the grouby function on it
data_df_cosim = pd.concat([data_df_cosim,data_df.id], axis =1)


In [4]:
#this puts the dataset into a matrix and indexed by the id
song_sim_matrix = data_df_cosim.groupby('id').sum()

song_sim_matrix.head()

,valence,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo
id,,,,,,,,,,,,
000G1xMMuwxNHmwVsBdtj1,0.555,0.0131,0.256,182347,0.895,0.000106,2,0.0821,-4.860,1,0.0707,191.307
000GyYHG4uWmlXieKLij8u,0.146,0.4690,0.797,180160,0.898,0.000000,1,0.0824,-5.922,0,0.5200,89.926
000Npgk5e2SgwGaIsN3ztv,0.494,0.9800,0.277,206972,0.145,0.879000,3,0.1110,-19.898,1,0.0845,75.644
000ZxLGm7jDlWCHtcXSeBe,0.854,0.7950,0.685,314667,0.483,0.878000,2,0.1130,-10.202,0,0.0337,97.694
000jBcNljWTnyjB4YO7ojf,0.969,0.6560,0.788,179747,0.808,0.000000,5,0.1540,-6.590,1,0.0395,113.046


In [5]:
#creating scaling function
def scaler(data):
    ss = StandardScaler()
    return ss.fit_transform(data)

#scale our dataframe
song_sim_matrix_scaled = scaler(song_sim_matrix)

#convert our numpy matrix into dataframe
song_sim_matrix_df = pd.DataFrame(song_sim_matrix_scaled, index=song_sim_matrix.index,columns=song_sim_matrix.columns)

In [6]:
# these dataframe is to print the results
data_df_show = data_df.copy()
song_sim_matrix_df_print = song_sim_matrix_df.copy()

## First Recomender Functions

In [7]:
# Use code from first recommender system
#getting the score 

def sim_score_generator(song,artist,number_of_recommedations):
    
    """takes in song and artist to return list of similar songs"""
    
    #takes artist and song name to find song id
    song_id = data_df[(data_df.main_artist == artist)&(data_df.name == song)]['id'].values[0]
    
    song_array = song_sim_matrix_df.loc[song_id].to_numpy()
    
    #tuple of sim score with song_index
    sim_score = []
    #counting the index
    song_index = -1
    
    #iterate through all songs in dataset
    for song in range(len(data_df)):
        
        song_index += 1

        # convert in to array
        other_song_array = song_sim_matrix_df.iloc[song].to_numpy()
        # sim score calculation
        score = dot(song_array,other_song_array)/(norm(song_array)*norm(other_song_array))
        
        #get other song id
        other_song_id = song_sim_matrix_df.iloc[song].name
        
        # append to list
        song_score_and_id = (score,song_index, other_song_id)
    
        # assign the position index of the song with the score
        sim_score.append(song_score_and_id)
    
    
    #list of N recommendations based on highest sim score and corresponding index
    return(sorted(sim_score,reverse=True)[1:number_of_recommedations+1])

def show_recommendations(top_recomendations):
    #unpack tuples in list of recommendation
    for score,song_idx,other_song_id in top_recomendations:
        #find the id of the from the sim matrix 
        
        sim_song_id = song_sim_matrix_df_print.iloc[song_idx].name
        
        
        #print the corresponding artist and song to that matches the id from the data_df
        print(f"Artist: {data_df_show[data_df_show.id.str.match(sim_song_id)]['main_artist'].values[0]}")
        print(f"Song: {data_df_show[data_df_show.id.str.match(sim_song_id)]['name'].values[0]}")
        print(f"Year & Popularity: {data_df_show[data_df_show.id.str.match(sim_song_id)]['year'].values[0]}, {data_df[data_df.id.str.match(sim_song_id)]['popularity'].values[0]}\n") 


## Playlist Functions

In [8]:
def create_playlist(song,artist):
    
    
    
    #find the first song id
    first_song_id = data_df[(data_df.main_artist == artist)&(data_df.name == song)]['id'].values[0]
    
    #append the song attributes to the playlist df
    playlist_df = (song_sim_matrix_df.loc[[first_song_id]])
    
    #drop the first song from the dataframes to prevent repeat recommendation
    #as this is a shortcut way method, the song_sim_matrix_df and data_df would need to be retransformed each time a new playlist is created to ensure songs are not missing
    song_sim_matrix_df.drop(first_song_id,inplace=True)
    data_df.drop(data_df[data_df.id.str.match(first_song_id)].index,inplace=True)
    data_df.reset_index(inplace=True,drop = True)
    
    return playlist_df

def add_song_playlist(song, playlist_df):
    
    """take in song from recommendations
    put into dataframe"""
    
    sim_score, song_idx, song_id = song
    
    #get pandas row of song
    song_row = song_sim_matrix_df.loc[[song_id]]
    
    #append to playlist dataframe
    playlist_df = playlist_df.append(song_row)
    
    
    #drop songs from dataframes if songs are in playlist to prevent repeat recommendation
    #as this is a shortcut way method, the song_sim_matrix_df and data_df would need to be retransformed to its original before dropping of songs
    song_sim_matrix_df.drop(song_id,inplace=True)
    data_df.drop(data_df[data_df.id.str.match(song_id)].index,inplace=True)
    data_df.reset_index(inplace=True,drop = True)
    
    return playlist_df


def playlist_reccomendations(playlist_df, number_of_recommedations):
    
    """takes in playlist dataframe and recommends n songs"""
    
    #create weights between 0.3 to 1
    if len(playlist_df) == 1:
        weights = np.array([1])
    else:
        weights = np.linspace(0.3,1,len(playlist_df))
    #reshape weight to be 2D
    weights = weights.reshape((len(weights),1))
    
    #convert playlist to an array
    playlist_array = playlist_df.to_numpy()
    
    #apply weights to playlist array
    adjusted_playlist_array = playlist_array*weights
    
    #sum all the song vectors and get average to be compared with other song vectors
    
    playlist_vector = sum(adjusted_playlist_array)/len(adjusted_playlist_array)
    
    #tuple of sim score with song_index
    sim_score = []
    #counting the index
    song_index = -1
    
    for song in range(len(data_df)):
        
        song_index += 1

        # convert in to array
        other_song_array = song_sim_matrix_df.iloc[song].to_numpy()
        # sim score calculation
        score = dot(playlist_vector,other_song_array)/(norm(playlist_vector)*norm(other_song_array))
        
        #get other song id
        other_song_id = song_sim_matrix_df.iloc[song].name
        
        # append to list
        song_score_and_id = (score,song_index, other_song_id)
    
        # assign the position index of the song with the score
        sim_score.append(song_score_and_id)
        
    
    
    #list of N recommendations based on highest sim score and corresponding index
    return(sorted(sim_score,reverse=True)[1:number_of_recommedations+1])

def show_current_playlist(playlist_df):
    
    """takes in playlist_df, and returns what is in the current playlist"""
    
    for song in playlist_df.index.to_list():
        
        #print the corresponding artist and song to that matches the id from the data_df
        print(f"Artist: {data_df_show[data_df_show.id.str.match(song)]['main_artist'].values[0]}")
        print(f"Song: {data_df_show[data_df_show.id.str.match(song)]['name'].values[0]}\n")

## Generating a Playlist for Survey

I'd like to make a playlist where I can conduct a survey to see if people do feel like the songs are similar from a user point of view. Up until now, we only know that the songs are statistically similar. 

### Survey version 1

In [9]:
#get Drake song ID (most of my friends like Drake)
song = "Look Alive (feat. Drake)"
artist = "BlocBoy JB"

In [10]:
#create new playlist_df
survey_playlist = create_playlist(song,artist)

In [11]:
#get recommendation for song Look Alive by BlocBoy

#return 10 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(survey_playlist,10)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Craig Morgan
Song: Almost Home
Year & Popularity: 2003, 60

Artist: Future
Song: Sleeping On The Floor
Year & Popularity: 2020, 73

Artist: Claude Debussy
Song: Children's Corner, L. 113: II. Jimbo's Lullaby. Assez modéré
Year & Popularity: 1946, 0

Artist: Paul McCartney
Song: Every Night - Live At Glasgow / 1979 / Remastered 2011
Year & Popularity: 1970, 27

Artist: NAV
Song: Trains (with Lil Keed)
Year & Popularity: 2020, 48

Artist: Killumantii
Song: Single (feat.Omeretta The Great)
Year & Popularity: 2018, 58

Artist: Chapterhouse
Song: Breather
Year & Popularity: 1990, 36

Artist: Los inquietos del vallenato
Song: Perdóname la Vida
Year & Popularity: 2000, 34

Artist: Bruce Springsteen
Song: Valentine's Day
Year & Popularity: 1987, 38

Artist: "Los Players"
Song: Destino
Year & Popularity: 2007, 46



In [12]:
#add song into playlist after looking at recommendation

survey_playlist = add_song_playlist(recommendations[2],survey_playlist)

#show survey_playlist
show_current_playlist(survey_playlist)

Artist: BlocBoy JB
Song: Look Alive (feat. Drake)

Artist: Lil Baby
Song: Woah



In [13]:
#return 10 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(survey_playlist,10)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Future
Song: Sleeping On The Floor
Year & Popularity: 2020, 73

Artist: The Larks
Song: Eyesight To the Blind
Year & Popularity: 1955, 3

Artist: NAV
Song: Trains (with Lil Keed)
Year & Popularity: 2020, 48

Artist: Los inquietos del vallenato
Song: Perdóname la Vida
Year & Popularity: 2000, 34

Artist: Эрих Мария Ремарк
Song: Часть 71.2 - На Западном фронте без перемен
Year & Popularity: 1929, 0

Artist: 21 Savage
Song: Snitches & Rats (feat. Young Nudy)
Year & Popularity: 2020, 71

Artist: Gucci Mane
Song: I Get the Bag (feat. Migos)
Year & Popularity: 2017, 73

Artist: 3OH!3
Song: STARSTRUKK
Year & Popularity: 2008, 46

Artist: Al Green
Song: For the Good Times
Year & Popularity: 1972, 48

Artist: Machine Gun Kelly
Song: LOCO
Year & Popularity: 2018, 62



In [14]:
#add song into playlist after looking at recommendation

survey_playlist = add_song_playlist(recommendations[7],survey_playlist)

#show survey_playlist
show_current_playlist(survey_playlist)

Artist: BlocBoy JB
Song: Look Alive (feat. Drake)

Artist: Lil Baby
Song: Woah

Artist: 3OH!3
Song: STARSTRUKK



In [15]:
#return 10 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(survey_playlist,10)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Future
Song: Tic Tac
Year & Popularity: 2020, 67

Artist: Gucci Mane
Song: I Get the Bag (feat. Migos)
Year & Popularity: 2017, 73

Artist: Saadettin Kaynak
Song: Gam Seni Terk Eylemez
Year & Popularity: 1927, 0

Artist: Astrud Gilberto
Song: Bim Bom
Year & Popularity: 1966, 30

Artist: Yo Gotti
Song: King Sh*t (feat. T.I.)
Year & Popularity: 2013, 54

Artist: The Four Lads
Song: Why Can't You Behave
Year & Popularity: 1953, 4

Artist: Kodak Black
Song: Coolin and Booted
Year & Popularity: 2017, 54

Artist: Mild High Club
Song: Note to Self
Year & Popularity: 2015, 56

Artist: NAV
Song: Trains (with Lil Keed)
Year & Popularity: 2020, 48

Artist: Los Baron De Apodaca
Song: La Culpa Fue Tuya
Year & Popularity: 1981, 25



In [16]:
#add song into playlist after looking at recommendation

survey_playlist = add_song_playlist(recommendations[3],survey_playlist)

#show survey_playlist
show_current_playlist(survey_playlist)

Artist: BlocBoy JB
Song: Look Alive (feat. Drake)

Artist: Lil Baby
Song: Woah

Artist: 3OH!3
Song: STARSTRUKK

Artist: Charli XCX
Song: Vroom Vroom



In [17]:
#return 15 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(survey_playlist,15)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Future
Song: Tic Tac
Year & Popularity: 2020, 67

Artist: Yo Gotti
Song: King Sh*t (feat. T.I.)
Year & Popularity: 2013, 54

Artist: Ian & Sylvia
Song: Darcy Farrow
Year & Popularity: 1965, 17

Artist: Gucci Mane
Song: I Get the Bag (feat. Migos)
Year & Popularity: 2017, 73

Artist: A$AP Ferg
Song: Jet Lag
Year & Popularity: 2019, 65

Artist: Lil Uzi Vert
Song: Venetia
Year & Popularity: 2020, 66

Artist: Future
Song: Sleeping On The Floor
Year & Popularity: 2020, 73

Artist: YoungBoy Never Broke Again
Song: Around
Year & Popularity: 2020, 64

Artist: The Four Lads
Song: Why Can't You Behave
Year & Popularity: 1953, 4

Artist: Mild High Club
Song: Note to Self
Year & Popularity: 2015, 56

Artist: NAV
Song: Trains (with Lil Keed)
Year & Popularity: 2020, 48

Artist: Avenged Sevenfold
Song: Beast and the Harlot
Year & Popularity: 2005, 62

Artist: Rodney Carrington
Song: Don't Look Now
Year & Popularity: 2003, 44

Artist: ОТВАЛ БАWKN
Song: Trap
Year & Popularity: 1941, 0

Artist:

Comments: I had to generate 15 recommendations instead as they dont seem to be recommending any niche content

In [18]:
#add song into playlist after looking at recommendation

survey_playlist = add_song_playlist(recommendations[13],survey_playlist)

#show survey_playlist
show_current_playlist(survey_playlist)

Artist: BlocBoy JB
Song: Look Alive (feat. Drake)

Artist: Lil Baby
Song: Woah

Artist: 3OH!3
Song: STARSTRUKK

Artist: Charli XCX
Song: Vroom Vroom

Artist: ОТВАЛ БАWKN
Song: Trap



In [19]:
#return 10 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(survey_playlist,15)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Future
Song: Tic Tac
Year & Popularity: 2020, 67

Artist: Yo Gotti
Song: King Sh*t (feat. T.I.)
Year & Popularity: 2013, 54

Artist: Dan Fogelberg
Song: Nexus
Year & Popularity: 1981, 25

Artist: Type O Negative
Song: Red Water (Christmas Mourning)
Year & Popularity: 1996, 38

Artist: Wreckx-N-Effect
Song: Rump Shaker - Radio Edit
Year & Popularity: 2002, 40

Artist: Ronan Hardiman
Song: Lord Of The Dance
Year & Popularity: 1996, 48

Artist: George Winston
Song: (Love Echoes in the) Pine Hills
Year & Popularity: 1981, 21

Artist: Adolescents
Song: Creatures
Year & Popularity: 1981, 32

Artist: Love
Song: ¡Que Vida!
Year & Popularity: 1966, 28

Artist: Blake Shelton
Song: Home (feat. Michael Bublé)
Year & Popularity: 2012, 54

Artist: Future
Song: Sleeping On The Floor
Year & Popularity: 2020, 73

Artist: Alejandro Escovedo
Song: Castanets
Year & Popularity: 2001, 41

Artist: Colm Wilkinson
Song: Who Am I?
Year & Popularity: 1987, 36

Artist: Meenakshi
Song: Balam Brahmachari
Ye

In [20]:
#add song into playlist after looking at recommendation

survey_playlist = add_song_playlist(recommendations[5],survey_playlist)

#show survey_playlist
show_current_playlist(survey_playlist)

Artist: BlocBoy JB
Song: Look Alive (feat. Drake)

Artist: Lil Baby
Song: Woah

Artist: 3OH!3
Song: STARSTRUKK

Artist: Charli XCX
Song: Vroom Vroom

Artist: ОТВАЛ БАWKN
Song: Trap

Artist: $uicideBoy$
Song: Aliens Are Ghosts ($UICIDEBOY$ X TRAVIS BARKER)



In [21]:
#return 10 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(survey_playlist,15)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: John Farnham
Song: Break The Ice
Year & Popularity: 1986, 41

Artist: Future
Song: Tic Tac
Year & Popularity: 2020, 67

Artist: Yo Gotti
Song: King Sh*t (feat. T.I.)
Year & Popularity: 2013, 54

Artist: Blake Shelton
Song: Home (feat. Michael Bublé)
Year & Popularity: 2012, 54

Artist: Future
Song: Sleeping On The Floor
Year & Popularity: 2020, 73

Artist: Sugar Chile Robinson
Song: Yancey Special
Year & Popularity: 1955, 2

Artist: Capital Cities
Song: Safe And Sound
Year & Popularity: 2013, 80

Artist: Sharpay Evans
Song: Fabulous
Year & Popularity: 2007, 59

Artist: The Rolling Stones
Song: Susie Q
Year & Popularity: 1964, 32

Artist: T-Wayne
Song: Nasty Freestyle
Year & Popularity: 2015, 64

Artist: Mitch Hedberg
Song: Sandwiches
Year & Popularity: 2003, 46

Artist: Wreckx-N-Effect
Song: Rump Shaker - Radio Edit
Year & Popularity: 2002, 40

Artist: Alejandro Escovedo
Song: Castanets
Year & Popularity: 2001, 41

Artist: Colm Wilkinson
Song: Who Am I?
Year & Popularity: 1987,

In [22]:
#add song into playlist after looking at recommendation

survey_playlist = add_song_playlist(recommendations[6],survey_playlist)

#show survey_playlist
show_current_playlist(survey_playlist)

Artist: BlocBoy JB
Song: Look Alive (feat. Drake)

Artist: Lil Baby
Song: Woah

Artist: 3OH!3
Song: STARSTRUKK

Artist: Charli XCX
Song: Vroom Vroom

Artist: ОТВАЛ БАWKN
Song: Trap

Artist: $uicideBoy$
Song: Aliens Are Ghosts ($UICIDEBOY$ X TRAVIS BARKER)

Artist: Tyga
Song: Ice Cream Man



In [23]:
#return 15 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(survey_playlist,15)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Future
Song: Tic Tac
Year & Popularity: 2020, 67

Artist: Yo Gotti
Song: King Sh*t (feat. T.I.)
Year & Popularity: 2013, 54

Artist: Wreckx-N-Effect
Song: Rump Shaker - Radio Edit
Year & Popularity: 2002, 40

Artist: John Farnham
Song: Break The Ice
Year & Popularity: 1986, 41

Artist: The Rolling Stones
Song: Susie Q
Year & Popularity: 1964, 32

Artist: Blake Shelton
Song: Home (feat. Michael Bublé)
Year & Popularity: 2012, 54

Artist: Peach Tree Rascals
Song: Mariposa
Year & Popularity: 2019, 83

Artist: Future
Song: Sleeping On The Floor
Year & Popularity: 2020, 73

Artist: Alejandro Escovedo
Song: Castanets
Year & Popularity: 2001, 41

Artist: Colm Wilkinson
Song: Who Am I?
Year & Popularity: 1987, 36

Artist: Sugar Chile Robinson
Song: Yancey Special
Year & Popularity: 1955, 2

Artist: Cal Tjader
Song: China Nights (Shina No Yoru) - 2nd Take
Year & Popularity: 1964, 14

Artist: T-Wayne
Song: Nasty Freestyle
Year & Popularity: 2015, 64

Artist: Pop Evil
Song: Monster You Ma

comments: After generating recommendations 7 times, the recommendations start to repeat themselves. Programatically, its possible to remove repeated recommendations after appearing a certain number of times. However this may be time consuming give the time constraint. This is something highlight as a limitation. 

Additionally, observing that most reccomendations tend to converge around 1 genre this may prove that a content based recommender will still lead you into a filter bubble, unless the user deliberately selects a genre that is new to them.

Trap songs generally have similar features. I will try to generate a playlist that have more variation of genres and have a potential to recommend a larger variation of genres. 

Irregardless, in order to validate that the music the recommender recommends is not only statistically similar, but also subjectively similar, this playlist will be sent out as a survey. The survey will also find out if there is general interest in a recommender that would POTENTIALLY recommend niche songs. 

### Survey Version 2

In [24]:
#get Drake song ID (most of my friends like Drake)
song = "Redbone"
artist = "Childish Gambino"

In [25]:
#create new playlist_df
survey_playlist = create_playlist(song,artist)

In [26]:
#return 10 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(survey_playlist,10)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: R.E.M.
Song: Driver 8
Year & Popularity: 1985, 36

Artist: Sade
Song: Kiss of Life
Year & Popularity: 1994, 60

Artist: Joan Baez
Song: Three Horses
Year & Popularity: 1971, 31

Artist: Los Alegres De Teran
Song: Los Tequileros
Year & Popularity: 1999, 45

Artist: The Band
Song: Long Black Veil - Remixed 2018
Year & Popularity: 1968, 20

Artist: The Shadows
Song: The Theme from 'The Deer Hunter' (Cavatina)
Year & Popularity: 1979, 46

Artist: Helen Forrest
Song: Taking A Chance On Love
Year & Popularity: 1956, 10

Artist: Frédéric Chopin
Song: Chopin: Nocturne No. 10 in A-Flat Major, Op. 32 No. 2
Year & Popularity: 1928, 12

Artist: Tindersticks
Song: Tiny Tears
Year & Popularity: 1995, 48

Artist: Francisco Canaro
Song: Que Sapa, Señor? - Remasterizado
Year & Popularity: 1931, 0



In [27]:
#add song into playlist after looking at recommendation

survey_playlist = add_song_playlist(recommendations[0],survey_playlist)

#show survey_playlist
show_current_playlist(survey_playlist)

Artist: Childish Gambino
Song: Redbone

Artist: Lil Wayne
Song: Lollipop



In [28]:
#return 10 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(survey_playlist,10)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: $atori Zoom
Song: Buster
Year & Popularity: 2019, 67

Artist: Эрих Мария Ремарк
Song: Часть 147.2 & Часть 148.1 - Триумфальная арка
Year & Popularity: 1945, 0

Artist: Stan Getz
Song: Jordu
Year & Popularity: 1958, 23

Artist: Paul Kantner
Song: Sketches of China - 2020 Remaster
Year & Popularity: 1973, 34

Artist: Exhorder
Song: Desecrator
Year & Popularity: 1990, 39

Artist: The Clovers
Song: Don't You Know I Love You
Year & Popularity: 1956, 13

Artist: The Shadows
Song: The Theme from 'The Deer Hunter' (Cavatina)
Year & Popularity: 1979, 46

Artist: Louis Hardin
Song: Symphonique #6 (Good For Goodie) - Instrumental
Year & Popularity: 1969, 23

Artist: Mehmet Kemiksiz
Song: Şu Boğaz Harbi Nedir
Year & Popularity: 1921, 0

Artist: "Booker T. & the M.G.s"
Song: Winter Wonderland
Year & Popularity: 1966, 15



In [29]:
#add song into playlist after looking at recommendation

survey_playlist = add_song_playlist(recommendations[2],survey_playlist)

#show survey_playlist
show_current_playlist(survey_playlist)

Artist: Childish Gambino
Song: Redbone

Artist: Lil Wayne
Song: Lollipop

Artist: Jill Scott
Song: The Way



In [30]:
#return 10 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(survey_playlist,10)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Эрих Мария Ремарк
Song: Часть 147.2 & Часть 148.1 - Триумфальная арка
Year & Popularity: 1945, 0

Artist: Delegation
Song: Love Is Like a Fire
Year & Popularity: 1978, 32

Artist: Mehmet Kemiksiz
Song: Şu Boğaz Harbi Nedir
Year & Popularity: 1921, 0

Artist: Dave Matthews Band
Song: Dancing Nancies
Year & Popularity: 1994, 44

Artist: Exhorder
Song: Desecrator
Year & Popularity: 1990, 39

Artist: "Booker T. & the M.G.s"
Song: Winter Wonderland
Year & Popularity: 1966, 15

Artist: Miles Davis
Song: 'Round Midnight
Year & Popularity: 1957, 57

Artist: Frank Sinatra
Song: Begin the Beguine
Year & Popularity: 1944, 14

Artist: Geeta Dutt
Song: O Mazam Raat Ni Tarli
Year & Popularity: 1948, 0

Artist: The Clovers
Song: Don't You Know I Love You
Year & Popularity: 1956, 13



In [31]:
#add song into playlist after looking at recommendation

survey_playlist = add_song_playlist(recommendations[8],survey_playlist)

#show survey_playlist
show_current_playlist(survey_playlist)

Artist: Childish Gambino
Song: Redbone

Artist: Lil Wayne
Song: Lollipop

Artist: Jill Scott
Song: The Way

Artist: Fito y Fitipaldis
Song: Soldadito marinero



In [32]:
#return 10 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(survey_playlist,10)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Information Society
Song: What's on Your Mind (Pure Energy)
Year & Popularity: 1988, 55

Artist: The Brobecks
Song: Small Cuts
Year & Popularity: 2009, 46

Artist: Silvia De Grasse
Song: La Morena Tumba Hombre
Year & Popularity: 1945, 0

Artist: Batmobile
Song: Transsylvanian Express
Year & Popularity: 1985, 33

Artist: XXXTENTACION
Song: Look At Me!
Year & Popularity: 2017, 82

Artist: Fred Waring
Song: White Christmas
Year & Popularity: 1957, 6

Artist: Carlos Santana
Song: A Love Supreme
Year & Popularity: 1973, 28

Artist: Head East
Song: City Of Gold
Year & Popularity: 1975, 22

Artist: Fleetwood Mac
Song: Doesn't Anything Last - Early Demo; 2004 Remaster
Year & Popularity: 1977, 36

Artist: Francisco Canaro
Song: Tristes Horas - Instrumental (Remasterizado)
Year & Popularity: 1929, 0



In [33]:
#add song into playlist after looking at recommendation

survey_playlist = add_song_playlist(recommendations[8],survey_playlist)

#show survey_playlist
show_current_playlist(survey_playlist)

Artist: Childish Gambino
Song: Redbone

Artist: Lil Wayne
Song: Lollipop

Artist: Jill Scott
Song: The Way

Artist: Fito y Fitipaldis
Song: Soldadito marinero

Artist: INOJ
Song: Time After Time



In [34]:
#return 10 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(survey_playlist,10)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Information Society
Song: What's on Your Mind (Pure Energy)
Year & Popularity: 1988, 55

Artist: Carmen Miranda
Song: Roda Pião
Year & Popularity: 1941, 0

Artist: Springtoifel
Song: Mainz Geht Unter
Year & Popularity: 1937, 0

Artist: Luther Vandross
Song: The Closer I Get to You (feat. Beyoncé Knowles)
Year & Popularity: 2003, 49

Artist: Francisco Canaro
Song: Canto - Remasterizado
Year & Popularity: 1934, 0

Artist: Halestorm
Song: I'm Not an Angel
Year & Popularity: 2009, 43

Artist: The Cleaners From Venus
Song: Corridor of Dreams
Year & Popularity: 1982, 44

Artist: Gene Ammons
Song: There Is No Greater Love
Year & Popularity: 1961, 37

Artist: Laura Ciffa
Song: Scream & Shout
Year & Popularity: 2013, 0

Artist: Ellen McCown
Song: Ode to Lola
Year & Popularity: 1951, 0



In [35]:
#add song into playlist after looking at recommendation

survey_playlist = add_song_playlist(recommendations[3],survey_playlist)

#show survey_playlist
show_current_playlist(survey_playlist)

Artist: Childish Gambino
Song: Redbone

Artist: Lil Wayne
Song: Lollipop

Artist: Jill Scott
Song: The Way

Artist: Fito y Fitipaldis
Song: Soldadito marinero

Artist: INOJ
Song: Time After Time

Artist: The Police
Song: Don't Stand So Close To Me



In [36]:
#return 10 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(survey_playlist,10)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Grant Green
Song: Green With Envy - Rudy Van Gelder Edition/24-Bit Mastering/2002 Digital Remaster
Year & Popularity: 1961, 20

Artist: Carmen Miranda
Song: Roda Pião
Year & Popularity: 1941, 0

Artist: Information Society
Song: What's on Your Mind (Pure Energy)
Year & Popularity: 1988, 55

Artist: Henry Mancini
Song: Moon River Cha Cha - Remastered
Year & Popularity: 1961, 11

Artist: Laura Ciffa
Song: Scream & Shout
Year & Popularity: 2013, 0

Artist: Barry Manilow
Song: Copacabana - Remastered
Year & Popularity: 1992, 44

Artist: Banda Vallarta Show
Song: Tu Pum Pum
Year & Popularity: 1993, 35

Artist: Ellen McCown
Song: Ode to Lola
Year & Popularity: 1951, 0

Artist: Cher
Song: Gypsys, Tramps & Thieves
Year & Popularity: 1971, 57

Artist: Iggy Pop
Song: I'm Bored
Year & Popularity: 1979, 38



In [37]:
#add song into playlist after looking at recommendation

survey_playlist = add_song_playlist(recommendations[9],survey_playlist)

#show survey_playlist
show_current_playlist(survey_playlist)

Artist: Childish Gambino
Song: Redbone

Artist: Lil Wayne
Song: Lollipop

Artist: Jill Scott
Song: The Way

Artist: Fito y Fitipaldis
Song: Soldadito marinero

Artist: INOJ
Song: Time After Time

Artist: The Police
Song: Don't Stand So Close To Me

Artist: Linda Ronstadt
Song: Poor, Poor Pitiful Me



The above playlist has enought varation of genres to test surveyees if thesongs are similar or not. Surveyees may have felt that the previous playlist natuarally sound the same because most of the songs fall in the same genre. 

Similarly, generating this playlist still ran in to the same problem where recommendations converged on the same songs. 

## Generating a Playlist for My Classmates

I plan to make a playlist for my classmates as this project would be the last we work on. This playlist would start with the graduation song by Vitamin C.

In [38]:
#get Vitamin C song ID
song = "Graduation (Friends Forever)"
artist = "Vitamin C"

In [39]:
#create new playlist
dsi_playlist = create_playlist(song,artist)

In [40]:
#get recommendation for song Graduation (Friends Forever) by Vitamin C

#return 10 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(dsi_playlist,10)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Orchestra Studio 7
Song: Mamma mia che vò sapè - Instrument and base Version
Year & Popularity: 1942, 0

Artist: Led Zeppelin
Song: Black Country Woman - 1993 Remaster
Year & Popularity: 1975, 44

Artist: Bryce Vine
Song: La La Land
Year & Popularity: 2019, 68

Artist: Average White Band
Song: Pick Up the Pieces
Year & Popularity: 1974, 55

Artist: Charles Givings
Song: Easy (LIke Sunday Morning)
Year & Popularity: 2003, 49

Artist: Javier Solís
Song: Las Mañanitas Tapatías
Year & Popularity: 1961, 17

Artist: Hüsker Dü
Song: Hardly Getting over It
Year & Popularity: 1986, 27

Artist: Beastie Boys
Song: B-Boys Makin' With The Freak Freak - Remastered 2009
Year & Popularity: 1994, 43

Artist: Hüsker Dü
Song: I Don't Know for Sure
Year & Popularity: 1986, 26

Artist: La Fiera de Ojinaga
Song: Te Vas a Acordar de Mi
Year & Popularity: 2018, 58



In [41]:
#add song into playlist after looking at recommendation

dsi_playlist = add_song_playlist(recommendations[7],dsi_playlist)

In [42]:
show_current_playlist(dsi_playlist)

Artist: Vitamin C
Song: Graduation (Friends Forever)

Artist: Chicago
Song: Will You Still Love Me? - 2009 Remaster



In [43]:
recommendations = playlist_reccomendations(dsi_playlist,10)

show_recommendations(recommendations)

Artist: The Who
Song: Love Reign O'er Me
Year & Popularity: 1979, 31

Artist: Taking Back Sunday
Song: One-Eighty By Summer
Year & Popularity: 2004, 44

Artist: Falling In Reverse
Song: Good Girls Bad Guys
Year & Popularity: 2011, 65

Artist: Wilco
Song: Red-Eyed and Blue - 2017 Remaster
Year & Popularity: 1996, 38

Artist: Paul Whiteman
Song: At Twilight
Year & Popularity: 1928, 0

Artist: Ludovico Sagese
Song: Colors of the Wind
Year & Popularity: 2019, 65

Artist: Louis Armstrong
Song: Bourbon Street Parade
Year & Popularity: 1960, 12

Artist: "Circus DJs"
Song: Cotton Eyed Joe
Year & Popularity: 2009, 49

Artist: Bon Jovi
Song: I'll Be There For You
Year & Popularity: 1988, 68

Artist: Ray Conniff
Song: Temptation
Year & Popularity: 1960, 36



In [44]:
#add song into playlist after looking at recommendation

dsi_playlist = add_song_playlist(recommendations[4],dsi_playlist)

show_current_playlist(dsi_playlist)

Artist: Vitamin C
Song: Graduation (Friends Forever)

Artist: Chicago
Song: Will You Still Love Me? - 2009 Remaster

Artist: Porter Robinson
Song: Divinity



In [45]:
#return 15 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(dsi_playlist,15)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Grupo Ladron
Song: Amor ausente
Year & Popularity: 1990, 30

Artist: Giasemi
Song: Exeis fridia san gefiria
Year & Popularity: 1950, 0

Artist: Roy Jones Jr.
Song: Body Head Anthem (feat. Mr. Magic & Choppa)
Year & Popularity: 2004, 54

Artist: Bon Jovi
Song: I'll Be There For You
Year & Popularity: 1988, 68

Artist: The J. Geils Band
Song: Homework - Live
Year & Popularity: 1972, 23

Artist: M. S. Subbulakshmi
Song: Unnuravum
Year & Popularity: 1939, 0

Artist: Sonny Boy Williamson II
Song: You Got To Give an Account
Year & Popularity: 1938, 5

Artist: Hlias Litos-Klarino
Song: Zagorisio-Xoros Hpeiroy
Year & Popularity: 1940, 0

Artist: Roy Jones Jr.
Song: Go Hard, Go Home (feat. Giz, Swellz & Choppa)
Year & Popularity: 2004, 59

Artist: The Royal Victoria Hotel Calypsos
Song: Go Down Emmanuel Road, Lady Oh
Year & Popularity: 1930, 1

Artist: Asha Bhosle
Song: Jadu Dale Hai Machal Machal
Year & Popularity: 1937, 0

Artist: Mr. Bungle
Song: The Air-Conditioned Nightmare
Year & 

In [46]:
#add song into playlist after looking at recommendation

dsi_playlist = add_song_playlist(recommendations[1],dsi_playlist)

show_current_playlist(dsi_playlist)

Artist: Vitamin C
Song: Graduation (Friends Forever)

Artist: Chicago
Song: Will You Still Love Me? - 2009 Remaster

Artist: Porter Robinson
Song: Divinity

Artist: Jhay Cortez
Song: Kobe En LA 2.0



In [47]:
#return 15 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(dsi_playlist,10)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: K.C.Dey
Song: Hari Ke Naam Bina Re
Year & Popularity: 1945, 0

Artist: DJ Luian
Song: La Ocasión
Year & Popularity: 2018, 67

Artist: Mohantara Talpade
Song: Gori Chunariya Ka Rang
Year & Popularity: 1947, 0

Artist: Bon Jovi
Song: I'll Be There For You
Year & Popularity: 1988, 68

Artist: Kausion
Song: Click, Click
Year & Popularity: 1995, 31

Artist: Smokie Norful
Song: Still Say Thank You
Year & Popularity: 2002, 36

Artist: Sonic Youth
Song: Sugar Kane
Year & Popularity: 1992, 54

Artist: Joan Sebastian
Song: Qué No Te Asombre
Year & Popularity: 1996, 47

Artist: Judas Priest
Song: A Touch of Evil
Year & Popularity: 1990, 48

Artist: Roy Jones Jr.
Song: Go Hard, Go Home (feat. Giz, Swellz & Choppa)
Year & Popularity: 2004, 59



In [48]:
#add song into playlist after looking at recommendation

dsi_playlist = add_song_playlist(recommendations[0],dsi_playlist)

show_current_playlist(dsi_playlist)

Artist: Vitamin C
Song: Graduation (Friends Forever)

Artist: Chicago
Song: Will You Still Love Me? - 2009 Remaster

Artist: Porter Robinson
Song: Divinity

Artist: Jhay Cortez
Song: Kobe En LA 2.0

Artist: Ying Yang Twins
Song: Get Low



In [49]:
#return 10 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(dsi_playlist,10)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Nina Simone
Song: Solitaire
Year & Popularity: 1959, 8

Artist: Los Inquietos Del Norte
Song: Mi Amigo el de Arriba
Year & Popularity: 2012, 63

Artist: Roy Jones Jr.
Song: Go Hard, Go Home (feat. Giz, Swellz & Choppa)
Year & Popularity: 2004, 59

Artist: Marcos Valle
Song: So Nice (Summer Samba)
Year & Popularity: 1968, 45

Artist: Judas Priest
Song: A Touch of Evil
Year & Popularity: 1990, 48

Artist: Bon Jovi
Song: I'll Be There For You
Year & Popularity: 1988, 68

Artist: Aretha Franklin
Song: Today I Sing the Blues - Mono Version
Year & Popularity: 1961, 11

Artist: Movie Sounds Unlimited
Song: Now Or Never - From High School Musical 3: Senior Year
Year & Popularity: 2009, 0

Artist: Héctor Acosta "El Torito"
Song: Sin Perdón
Year & Popularity: 2008, 48

Artist: Lil Uzi Vert
Song: Early 20 Rager
Year & Popularity: 2017, 55



In [50]:
#add song into playlist after looking at recommendation

dsi_playlist = add_song_playlist(recommendations[7],dsi_playlist)

show_current_playlist(dsi_playlist)

Artist: Vitamin C
Song: Graduation (Friends Forever)

Artist: Chicago
Song: Will You Still Love Me? - 2009 Remaster

Artist: Porter Robinson
Song: Divinity

Artist: Jhay Cortez
Song: Kobe En LA 2.0

Artist: Ying Yang Twins
Song: Get Low

Artist: Gary Numan
Song: M.E.



In [51]:
#return 15 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(dsi_playlist,15)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Judas Priest
Song: A Touch of Evil
Year & Popularity: 1990, 48

Artist: "Billy Crash Craddock"
Song: Rub It In
Year & Popularity: 1996, 39

Artist: U2
Song: Red Hill Mining Town - Steve Lillywhite 2017 Mix
Year & Popularity: 1987, 36

Artist: Marcos Valle
Song: So Nice (Summer Samba)
Year & Popularity: 1968, 45

Artist: Shamshad Begum
Song: Nainon Ke Baan Ki Reet Anokhi
Year & Popularity: 1941, 0

Artist: The Housemartins
Song: Get up off Our Knees
Year & Popularity: 1986, 24

Artist: Santana
Song: Treat
Year & Popularity: 1969, 43

Artist: Roy Jones Jr.
Song: Go Hard, Go Home (feat. Giz, Swellz & Choppa)
Year & Popularity: 2004, 59

Artist: Chocolate Milk
Song: Doc
Year & Popularity: 1979, 41

Artist: Doris Day
Song: Zip-A-Dee-Doo-Dah (with Jimmy Joyce & His Children's Chorus)
Year & Popularity: 1960, 10

Artist: A Guy Called Gerald
Song: Voodoo Ray
Year & Popularity: 1988, 48

Artist: Smokie Norful
Song: Still Say Thank You
Year & Popularity: 2002, 36

Artist: Bon Jovi
Song: 

In [52]:
#add song into playlist after looking at recommendation

dsi_playlist = add_song_playlist(recommendations[14],dsi_playlist)

show_current_playlist(dsi_playlist)

Artist: Vitamin C
Song: Graduation (Friends Forever)

Artist: Chicago
Song: Will You Still Love Me? - 2009 Remaster

Artist: Porter Robinson
Song: Divinity

Artist: Jhay Cortez
Song: Kobe En LA 2.0

Artist: Ying Yang Twins
Song: Get Low

Artist: Gary Numan
Song: M.E.

Artist: PJ Harvey
Song: We Float



In [53]:
#return 15 similar song with raw similiarity scores and assign into list 
recommendations = playlist_reccomendations(dsi_playlist,15)

#print similar songs with corresponding artist and song name
show_recommendations(recommendations)

Artist: Sandi Patty
Song: Was It a Morning Like This
Year & Popularity: 1986, 31

Artist: Judas Priest
Song: A Touch of Evil
Year & Popularity: 1990, 48

Artist: Sade
Song: Paradise
Year & Popularity: 1994, 58

Artist: Rush
Song: Anagram - 2004 Remaster
Year & Popularity: 1989, 28

Artist: Ivan Gough
Song: In My Mind (feat. Georgi Kay) - Axwell Radio Edit
Year & Popularity: 2012, 61

Artist: Beastie Boys
Song: Hey Ladies
Year & Popularity: 1989, 53

Artist: Los Bukis
Song: A Aquella
Year & Popularity: 1993, 44

Artist: Blitz
Song: Voice Of A Generation
Year & Popularity: 1982, 31

Artist: Pat Green
Song: Texas On My Mind
Year & Popularity: 2001, 42

Artist: Roy Jones Jr.
Song: Go Hard, Go Home (feat. Giz, Swellz & Choppa)
Year & Popularity: 2004, 59

Artist: Stelios Kazantzidis
Song: Isos aurio
Year & Popularity: 1950, 0

Artist: Tribal Seeds
Song: Island Girl
Year & Popularity: 2008, 41

Artist: Joan Sebastian
Song: Como Tú Decidas
Year & Popularity: 1998, 57

Artist: Me First and the

In [54]:
#add song into playlist after looking at recommendation

dsi_playlist = add_song_playlist(recommendations[10],dsi_playlist)

show_current_playlist(dsi_playlist)

Artist: Vitamin C
Song: Graduation (Friends Forever)

Artist: Chicago
Song: Will You Still Love Me? - 2009 Remaster

Artist: Porter Robinson
Song: Divinity

Artist: Jhay Cortez
Song: Kobe En LA 2.0

Artist: Ying Yang Twins
Song: Get Low

Artist: Gary Numan
Song: M.E.

Artist: PJ Harvey
Song: We Float

Artist: R.E.M.
Song: Camera - Original Album Version



And thats it! This will be the 8 song in that I would base my DSI 21 Grad playlist on!!